In [1]:
#pip install -U scikit-learn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

#needed for openai gym environment
import gym
import gym_anytrading

#needed for reinforcement learning algorithms

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN
from stable_baselines3 import PPO

from sklearn.model_selection import ParameterGrid


In [3]:
googl_df = pd.read_csv('GOOGL.csv')
googl_df = googl_df.drop(columns=['Dividends', 'Stock Splits'])
googl_df['N_date'] = googl_df['Date'].str[:10]
googl_df['N_date'] = pd.to_datetime(googl_df['N_date'])
googl_df = googl_df.drop(columns=['Date'])

In [4]:
googl_df.rename(columns = {'N_date':'Date'}, inplace=True)
googl_df.set_index('Date', inplace=True)
googl_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2004-08-19,2.502503,2.604104,2.401401,2.511011,893181924
2004-08-20,2.527778,2.729730,2.515015,2.710460,456686856
2004-08-23,2.771522,2.839840,2.728979,2.737738,365122512
2004-08-24,2.783784,2.792793,2.591842,2.624374,304946748
2004-08-25,2.626627,2.702703,2.599600,2.652653,183772044


In [5]:
googl_maker = lambda: gym.make('stocks-v0', df=googl_df, frame_bound=(700,3500), window_size=30)
# can try this with other types of enivornment not just the DummyVec
googl_env = DummyVecEnv([googl_maker])

### Hypertuning for DQN -

In [6]:
dqn_model = DQN('MlpPolicy', googl_env, verbose=1)
dqn_model.learn(total_timesteps=25000)

Using cpu device
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 13107    |
|    time_elapsed     | 0        |
|    total_timesteps  | 11196    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 13311    |
|    time_elapsed     | 1        |
|    total_timesteps  | 22392    |
----------------------------------


In [7]:
googl_env = gym.make('stocks-v0', df=googl_df, frame_bound=(3600,3650), window_size=30)
obs = googl_env.reset()
while True:
    obs = obs[np.newaxis, ...]
    action, _states = dqn_model.predict(obs)
    obs,rewards, done, info = googl_env.step(action)
    if done:
        print('info', info)
        break

info {'total_reward': 1.66400146484375, 'total_profit': 1.0539949366332015, 'position': 1}


### HyperTuning for  PPO

In [8]:
#ppo_model = PPO('MlpPolicy', googl_env, verbose=1)
#ppo_model.learn(total_timesteps=25000)

In [9]:
#googl_env = gym.make('stocks-v0', df=googl_df, frame_bound=(3600,3650), window_size=30)
#obs = googl_env.reset()
#while True:
#    obs = obs[np.newaxis, ...]
#    action, _states = ppo_model.predict(obs)
#    obs,rewards, done, info = googl_env.step(action)
#    if done:
#        print('info', info)
#        break